In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import seaborn as sns
sys.path.append(os.path.abspath('../sc'))
import preprocess
import utils
import phonenumbers
import nltk
from IPython.display import display, HTML
import traceback
from nltk.tag.stanford import StanfordNERTagger
from nltk.tree import Tree
import re
import tqdm
csvs = ["../d/train.csv", "../d/dev.csv", "../d/test.csv", "../d/val.csv"]
#csvs = ["../d/val.csv"]
df_raw = pd.concat([pd.read_csv(p, usecols=["id","question1", "question2"]) for p in csvs])
import gensim
import nltk
m = gensim.models.KeyedVectors.load_word2vec_format("../d/GoogleNews-vectors-negative300.bin", binary=True)
vocabs = set(m.vocab.keys())

In [8]:

def prep(text, vocabs):
    text = re.sub("’", "'", text)    
    text = re.sub("`", "'", text) # special single quote
    text = re.sub("“", '"', text) # special double quote
    text = re.sub("？", "?", text) 
    text = re.sub("…", " ", text) 
    text = re.sub("é", "e", text)
    text = re.sub(r"\.+", ".", text)
    text = text.replace("[math]", " ")
    text = text.replace("[/math]", " ")
    
    if prep.dict is None:
        prep.dict = pd.read_csv("../d/replace.csv").set_index("src")["dst"].to_dict()
        for k in prep.dict.keys():
            prep.dict[k.lower()] = prep.dict[k]
        
    for k in prep.dict.keys():
        if k in text:
            text = text.replace(k, prep.dict[k])
    # numbers
    #re.findall(r"\d+\.\d+","I hear 10.55Ghz sounds. ")

    if prep.units is None:
        with open("../d/units.csv", "rU") as f:
            prep.units = [l.replace('\n','') for l in f]

    for u in prep.units:
        text = re.sub(r"(\d+\.\d+){0}".format(u),"\\1 {0}".format(u), text)
    
    matches = re.finditer(r"([a-zA-z]*)\.([a-zA-z]*)", text)
    for match in matches:
    #if matches is not None:
        m01 = match.group(0)
        m0 = match.group(1)
        m1 = match.group(2)
        if m01 not in vocabs and m1 in vocabs \
                            and m1.lower() not in ("com", "org","net","exe", "js", "biz", "care", "ly", "io", "in", "jp", "au", "gov", "ca", "cn", "fr", "hk", "kr", "mx") \
                            and (m1.lower() in ("i","a") or len(m1) > 1) \
                            and m1[-1] != ".":
            #print text
            text = text.replace(m01, m01.replace(".", ". "))
            #print text
    matches = re.finditer(r"([a-zA-z]*)?([a-zA-z]*)", text)
    for match in matches:
    #if matches is not None:
        m01 = match.group(0)
        m0 = match.group(1)
        m1 = match.group(2)
        if m01 not in vocabs and m0 in vocabs \
                            and m1 in vocabs \
                            and (m1.lower() in ("i","a") or len(m1) > 1):
            text = text.replace(m01, m01.replace(".", ". "))
            
    #text = re.sub(r" (?=[\w]*)/(?=[\w]*) ", " or ", text)
    text = re.sub(r"/", " or ", text)    
    return text
prep.dict = None
prep.units = None

def post(tokens):
    out = []
    for t in tokens:
        if t[-1] == ".":
            out.append(t[:-1])
        else:
            out.append(t)
    return out
        
def extract_ne(chunked):
    prev = None
    continuous_chunk = []
    current_chunk = []
    for i in chunked:
        if type(i) == Tree:
            current_chunk.append(" ".join([token for token, pos in i.leaves()]))
        elif current_chunk:
            named_entity = " ".join(current_chunk)
            if named_entity not in continuous_chunk:
                continuous_chunk.append(named_entity)
                current_chunk = []
            else:
                continue
    return continuous_chunk

df = df_raw.copy()
df.fillna(value="", inplace=True)
count = 0
words = set([])
t_failed = []
s_failed = []
failed=[]
# for s1, s2 in tqdm.tqdm(zip(df.question1.values, df.question2.values)):
#     t = nltk.word_tokenize(prep(s1.decode("utf-8"), vocabs).encode("ascii","ignore"))
#     f = False
#     for w in t:
#         if w not in m:
#             words.add(w)
#             f = True
#     if f:
#         t_failed.append(t)
#         s_failed.append(s1)
#     count += 1
#     if count > 100000:
#         break

for s1, s2 in tqdm.tqdm(zip(df.question1.values, df.question2.values)):
    try:
        t = post(nltk.word_tokenize(prep(s1.decode("utf-8"), vocabs).encode("ascii","ignore")))
        f = False
        for w in t:
            if w not in m:
                words.add(w)
                f = True
        if f:
            t_failed.append(t)
            s_failed.append(s1)
    except:
        print s1
        print "-" * 15        
        failed.append(s1)

    try:
        t = post(nltk.word_tokenize(prep(s2.decode("utf-8"), vocabs).encode("ascii","ignore")))
        f = False
        for w in t:
            if w not in m:
                words.add(w)
                f = True
        if f:
            t_failed.append(t)
            s_failed.append(s2)
    except:
#        traceback.print_exc()
        print s2
        print "-" * 15        
        failed.append(s2)
    count += 1
    #if count > 100000:
    #    break



  0%|          | 1297/2750086 [00:01<54:52, 834.90it/s]

  0%|          | 11830/2750086 [00:14<55:19, 824.79it/s]


  1%|          | 23685/2750086 [00:28<56:21, 806.17it/s]


  1%|▏         | 35414/2750086 [00:41<54:17, 833.37it/s]


  2%|▏         | 47117/2750086 [00:55<54:19, 829.16it/s]


  2%|▏         | 58740/2750086 [01:09<53:58, 831.02it/s]


...

 98%|█████████▊| 2703636/2750086 [42:21<00:41, 1107.34it/s]


 99%|█████████▉| 2718516/2750086 [42:34<00:27, 1146.50it/s]


 99%|█████████▉| 2732947/2750086 [42:47<00:19, 883.30it/s]


100%|█████████▉| 2744730/2750086 [43:01<00:06, 871.78it/s]


100%|██████████| 2750086/2750086 [43:07<00:00, 1062.77it/s]

In [37]:
print "Rs" in m
m.similarity("Rs","USD")
# print len(words)
# print len(t_failed)
# print len(s_failed)
# print t_failed[:5]

True


0.55803175068636146

In [9]:
ws=set([])
seconds = []
for w in words:
    if "." in w and not w.endswith(".com") and not w.endswith(".org") and not w.endswith(".net"):
        ws.update([w])
        #l = w.split(".")
        #if len(l) > 1:
        #    seconds.append(l[1])
#print set(seconds)

In [12]:
ws

{'9.5+',
 'D.M.M',
 'IELTS:6.5',
 'I.V',
 'D3.JS',
 'livepro.in',
 'I.N',
 '3.06:1',
 '16.50',
 'c=-3.25',
 'I.G',
 'B.a',
 'I.B',
 'I.C',
 'justeat.in',
 'IO.js',
 '1.15',
 '1.12',
 ',5.12',
 'F1.7',
 '7.5x10-11C',
 'F1.8',
 '99.100',
 '7.62x54R',
 '1.18',
 'B.f',
 'I.m',
 '3^0.5',
 'no.s',
 "'Honolulu.to",
 '153.5',
 '262.5',
 '4X1.7',
 'www.socialchaal',
 'R.T',
 '-0.597',
 'R.V.X',
 '45.8',
 '1.1_',
 'www.laracasTCom',
 '45.0',
 'ACMA_6.1',
 '.e^',
 '\\Python27\\python.exe',
 'R.o',
 '171.5',
 '700.00',
 '+.+\\sin^2',
 'D.E',
 'D.D',
 '1070.67',
 'L.R-99.5',
 '1.1s',
 'D.B',
 'D.M',
 '3.5-5.6',
 'D.O',
 'D.N',
 '.and',
 'D.J',
 'D.U',
 'n.c.r',
 'condo.h',
 'p-0.02',
 'D.Y',
 'H.e.l.p.l',
 '1.1i',
 'B.L.S',
 'IIS7.0',
 '3.568568568568',
 'H.BEcon',
 '96.53',
 'friend.ly',
 '802.1x',
 'W.SMaugham',
 'server.js',
 '8.0.6.0',
 '01.08.2016',
 '9.50',
 '.fxml',
 '10th-9.2',
 '399.00',
 'Xth-77.2',
 'after1a.m',
 'J.E.E',
 'amzn.in',
 'B.N',
 '1.998',
 '1.ktm',
 '.45cal',
 '1^n+2^n+3^n+.

In [53]:
st=m.vocab.keys()
print st[:5]

[u'Allanah_Munson', u'WINDS_WILL', u'nab_sexual_predators', u'By_Alexandra_Barham', u'Mayor_Noramie_Jasmin']
